In [ ]:
library(randomForest)
library(caret)
#replace audris with your netid
dat=read.table("/home/smishr11/features.csv", sep=";")
names(dat)=c("state","fruit","path", paste("f", 1:2048, sep=""))


In [ ]:
table(as.character(dat$state)) 
      
-sort(-table(as.character(dat$fruit)))



 fresh rotten 
     6    938 


       endive  strawberries      tomatoes summer_squash      tangelos 
          438            72            72            54            54 
     rutabaga    tangerines       spinach    tomatillos          yams 
           42            42            24            24            24 
       sapote       turnips   raspberries       rhubarb        squash 
           18            18            14            12            12 
      lettuce     scallions   swiss_chard    watercress 
            6             6             6             6 

In [ ]:
#Lets try to see if we have fresh or not, e.g., quality control for groceries:
# This specific assignment investigates predictor for rotten endive
resp = as.factor(dat$state=='rotten'&dat$fruit=='endive')
grps = createDataPartition(resp,p=.7,list=FALSE)
trns = grps
vals = -grps
#lets use all 2048 feature vectors for the model
rng=1:2048
rng=1:20
trains <- dat[trns,]
validates  <- dat[vals,]
trainsX <-trains[,-c(1:3)][,rng] 
trainsY <- resp[trns];
valsX <-validates[,-c(1:3)][,rng] 
valsY <- resp[vals];

#No we can fit the model (Random Forest) and predict on the validation set (as the number of predictors increases, TF becomes much slower, so we use just 100 out of 248*6 predictors here:

rfs=randomForest(trainsX[,],y=trainsY)
prfs.v = predict(rfs,valsX)
table(prfs.v,valsY)

       valsY
prfs.v  FALSE TRUE
  FALSE   140   32
  TRUE     11   99

Discuss the finding....

The model chooses "false" more often than "true". This makes sense, as most of the images it processes are likely not what it is looking for. 

In [ ]:
# 200 predictors (10%) explain XX% of the variance
pca <- prcomp(dat[,-c(1:3)], center = TRUE, scale = TRUE) 
sum(pca$sdev[1:200])/sum(pca$sdev)


[1] 0.6073204

In [ ]:


trainsRX = pca$x[trns,1:200]
valsRX <-pca$x[vals,1:200] 
rfsR=randomForest(trainsRX,y=trainsY)
prfsR.v = predict(rfsR,valsRX)
table(prfsR.v,valsY)



       valsY
prfsR.v FALSE TRUE
  FALSE   145    8
  TRUE      6  123

Discuss the finding....

The model marginally improves, meaning that PCA is an effective way to improve model predictions. 


In [ ]:
library(e1071)
svmMod = svm(trainsY~., data = trainsRX)
prfsS.v = predict(svmMod,valsRX)
table(prfsS.v,valsY)


       valsY
prfsS.v FALSE TRUE
  FALSE   144    0
  TRUE      7  131

Discuss the finding

This one performs the best- it is most confident and definitively chooses False/False often. 

Finally, lets try the oldest/simplest technique - k-Nearedt Neighbor (knn)


In [ ]:
#Looks like it take a long time
library(class)
trainsKX = dat[trns,-c(1:3)]
valsKX = dat[vals,-c(1:3)] 
trainsY=resp[trns]
valsY=resp[vals]
kMod = knn(trainsKX, valsKX, trainsY)
table(kMod,valsY)


       valsY
kMod    FALSE TRUE
  FALSE   151    0
  TRUE      0  131

Discuss the finding

In [ ]:
This model is by far the best. While the model may be filing images incorrectly, it is very confident about how it is filing them, with every image either false/false or true/true.